In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
%matplotlib inline
import torch
from deepseries.models import Wave2WaveV1
from deepseries.dataset import Property, TimeSeries, Seq2SeqDataLoader
from deepseries.nn.loss import MSELoss, RMSELoss
from deepseries.train import Learner
from deepseries.optim import ReduceCosineAnnealingLR
import deepseries.functional as F
from deepseries.analysis import SeriesAnalysisModel
from torch.optim import Adam
from torch import nn
import matplotlib as mpl
from sklearn.metrics import mean_absolute_error
import chinese_calendar as calendar
import datetime as dt

C:\Users\evilp\AppData\Local\Continuum\anaconda3\envs\dl\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\evilp\AppData\Local\Continuum\anaconda3\envs\dl\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\evilp\AppData\Local\Continuum\anaconda3\envs\dl\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\evilp\AppData\Local\Continuum\anaconda3\envs\dl\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\evilp\AppDa

In [2]:
# util function

def normalize(x, axis, fill_zero=True):
    mu = np.nanmean(x, axis, keepdims=True)
    std = np.nanstd(x, axis, keepdims=True)
    x_norm = (x - mu) / std
    if fill_zero:
        x_norm = np.nan_to_num(x_norm)
    return x_norm, mu, std

In [3]:
power = pd.read_csv('../data/df.csv', parse_dates=['data_time'])[['data_time', 'cid', 'value']]
power = power.set_index("data_time").groupby("cid").resample("1H").sum().reset_index()
power = power.pivot(index='cid', columns='data_time', values='value')
power = power.apply(np.log1p)

In [4]:
N_TEST = 24 * 30
N_VALID = 24 * 2
DROP_ZERO = True
DEC_LEN = 24 * 2
ENC_LEN = 24 * 7
time_free_space = 24

drop_before = 15000

data_time,2017-04-30 00:00:00,2017-04-30 01:00:00,2017-04-30 02:00:00,2017-04-30 03:00:00,2017-04-30 04:00:00,2017-04-30 05:00:00,2017-04-30 06:00:00,2017-04-30 07:00:00,2017-04-30 08:00:00,2017-04-30 09:00:00,...,2020-04-15 14:00:00,2020-04-15 15:00:00,2020-04-15 16:00:00,2020-04-15 17:00:00,2020-04-15 18:00:00,2020-04-15 19:00:00,2020-04-15 20:00:00,2020-04-15 21:00:00,2020-04-15 22:00:00,2020-04-15 23:00:00
cid,,,,,,,,,,,,,,,,,,,,,
91E0B6DC00B41783E0530BC8100A0E30,9.292928,9.3268,9.30424,9.267823,9.177518,9.093689,8.976601,9.014201,9.044114,9.073157,...,9.656942,9.643891,9.650329,9.694016,9.631585,9.558167,9.479534,9.326953,8.629712,8.505145
91E0B6DC00B51783E0530BC8100A0E30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9.354836,9.331511,9.381401,9.448442,9.514657,9.566265,9.534905,9.418885,8.291041,7.909200
91E0B6DC00B91783E0530BC8100A0E30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9.930853,9.949363,9.728605,9.586129,9.539908,9.841113,9.958179,9.932017,9.902171,9.840213
91E0B6DC00BA1783E0530BC8100A0E30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,10.379793,10.371832,10.354687,10.408397,10.402454,10.351226,10.370210,10.380129,10.355272,10.406317
91E0B6DC00BB1783E0530BC8100A0E30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9.521224,9.497918,9.521278,9.539263,9.578105,9.593460,9.548733,9.278921,8.203205,7.784954
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91E0C8BAA1C5259EE0530BC8100AFFB2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.125358,8.185542,8.277891,8.384448,8.457571,8.576566,8.546124,8.341388,8.069493,7.734396
91E0C8BAA1CB259EE0530BC8100AFFB2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9.375100,9.370061,9.292102,9.188006,9.164084,6.443777,6.306674,6.239941,6.203989,6.186016
91E1E413742C0582E0530BC8100A3E6E,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.481430,8.343510,8.441935,8.352385,7.711793,7.623155,7.764489,7.604585,7.787220,8.488946
